In [1]:
import sys

sys.path.append("..")

from src.auth.aurora import get_portal_client

In [2]:
portal_client = get_portal_client()

In [3]:
df = portal_client.query_to_pandas(
    """SELECT
        DISTINCT test.id AS test_id
    FROM topaz.api_service_tests AS test
    INNER JOIN topaz.api_mode_changes AS mode_change
    ON test.mode_change_id = mode_change.id
    INNER JOIN topaz.api_sites AS site
    ON test.site_id = site.id
    INNER JOIN topaz.api_toc_rows AS toc
    ON (mode_change.toc_id = toc.toc_id)
    AND (
        (test.domain = toc.domain) OR
        (toc.domain IS NULL)
    )
    AND (
        (test.panel = toc.panel) OR
        (toc.panel IS NULL)
    )
    AND (
        (test.loop = toc.loop) OR
        (toc.loop IS NULL)
    )
    AND (test.address = toc.address)
    AND (test.address_type = toc.address_type)
    AND (
        (
            (
                (test.sub_address = '') OR
                (test.sub_address IS NULL)
            ) AND (
                (toc.sub_address IS NULL) OR
                (toc.sub_address = 0)
            )
        ) OR
        (test.sub_address = toc.sub_address)
    )
    INNER JOIN topaz.api_service_tasks AS task_template
    ON task_template.template_id = site.template_id
    AND (
        (task_template.asset_type = toc.lcs_device_type)
        OR (task_template.asset_type = '')
        OR (
            site.protocol_id = 34
            AND asset_type = 'el'
        )
        OR (
            site.protocol_id = 35
            AND asset_type = 'ext'
        )
    )
    AND test.created_at BETWEEN task_template.created_at AND IFNULL(task_template.deleted_at, CURRENT_TIMESTAMP())
    AND task_template.type IN (1,3)
    LEFT JOIN topaz.api_service_tasks_record AS task_record
    ON task_record.task_id = task_template.id
    AND task_record.test_id = test.id
    WHERE task_record.id IS NULL
    LIMIT 1
    """
)

In [4]:
df

,test_id
0,163291
